#### Задание:
Разработка системы транслитерации/транскрипции для существующего естественного языка. Трансдьюсер должен быть нужен для транслитерации.

Выполняется транслитерация **лесного ненецкого языка с дополненной кириллицы в латиническую запись**.

Основана на транскрипции, используемой лесной ненецкой экспедицией (в пуровский диалект), составленной по результатам полевой работы + докладу И. Егорова (по нумтовскому диалекту л.-н. языка). За кириллическую запись берётся вариант, представленный в словаре лесного ненецкого гласного Бармич-Вэлло 2022 (в котором представлены и долгие маркированные, и краткие маркированные, и немаркированные гласные + написание гортанной смычки в двух вариантах).

**Гласные:**

![Гласные в файле vocal5!](vocal5.png)

**Согласные**

Палатализуются перед передними гласными: е, ё, ю, я, и. Веляризованного "д" нет: это алломорф "й". Йот не представлен из-за неразличения кириллического обозначения йота и и краткого. Также есть мягкий знак как показатель палатализации.

![Согласные в файле cons](cons.png)

#### Задаём функцию переходов и функцию транслитерации.

In [5]:
import re

Пусть у нас будет 4 состояния: 
* нулевое - входное, в котором способен обработаться весь алфавит первого символа (гласные переднего ряда в нём не нужны, поскольку они демонстрируют палатализованность согласного, следовательно, не бывают в начале слова)
* palat, в котором согласные проверяют наличие дальше палатализующего гласного
* checkh, в котором смычка выбирает алломорф
* length, в котором гласные проверяются на долготу

In [89]:
emp = ""
transition = {
    ("q0", f"%:{emp}"): 'q0',  # consonants return their corresponding Latin characters, checking most of them as palatilized
    ("q0", f"$:{emp}"): 'q0',
    ("q0", "п:p"): 'palat',
    ("q0", "т:t"): 'palat',
    ("q0", "ч:č"): 'q0',  # doesn't have a palatalized corresponding character
    ("q0", "к:k"): 'palat',
    ("q0", "м:m"): 'palat',
    ("q0", "н:n"): 'palat',
    ("q0", "ӈ:ŋ"): 'palat',
    ("q0", "с:s"): 'palat',
    ("q0", "ш:sʹ"): 'q0',  # doesn't have a palatalized corresponding character
    ("q0", "х:x"): 'palat',
    ("q0", "ԯ:λ"): 'palat',
    ("q0", "в:w"): 'palat',
    ("q0", "л:l"): 'palat',
    ("q0", "д:dʹ"): 'q0',  # doesn't have a velarized corresponding character
    ("q0", f"':{emp}"): 'checkh',  # cheking if there's a ʰ
    ("q0", f'":{emp}'): 'checkh',
    ("q0", "а:a"): 'length',  # checking most of the vowels for length; æ can't be long, it is mostly unmarked
    ("q0", "о:o"): 'length',
    ("q0", "э:e"): 'length',
    ("q0", "у:u"): 'length',
    ("q0", "ы:i"): 'length',
    ("q0", "ӹ:æ̆"): 'q0',
    ("q0", "ӭ:æ"): 'q0',
    ("q0", " : "): 'q0',  # making whitespaces

    ("palat", "ь:ʹ"): "q0",  # checking for palatilization
    ("palat", "е:ʹe"): 'length',
    ("palat", "ё:ʹo"): 'length',
    ("palat", "ю:ʹu"): 'length',
    ("palat", "я:ʹa"): 'length',
    ("palat", "и:ʹi"): 'length',
    ("palat", " : "): 'q0',
    ("palat", "п:p"): 'palat',
    ("palat", "т:t"): 'palat',
    ("palat", "ч:č"): 'q0',
    ("palat", "к:k"): 'palat',
    ("palat", "м:m"): 'palat',
    ("palat", "н:n"): 'palat',
    ("palat", "ӈ:ŋ"): 'palat',
    ("palat", "с:s"): 'palat',
    ("palat", "ш:sʹ"): 'q0',
    ("palat", "х:x"): 'palat',
    ("palat", "ԯ:λ"): 'palat',
    ("palat", "в:w"): 'palat',
    ("palat", "л:l"): 'palat',
    ("palat", "д:dʹ"): 'q0',
    ("palat", f"':{emp}"): 'checkh',
    ("palat", f'":{emp}'): 'checkh',

    ("palat", "а:a"): 'length',
    ("palat", "о:o"): 'length',
    ("palat", "э:e"): 'length',
    ("palat", "у:u"): 'length',
    ("palat", "ы:i"): 'length',
    ("palat", "ӹ:æ̆"): 'q0',
    ("palat", "ӭ:æ"): 'q0',
    ("palat", " : "): 'q0',
    ("palat", f"%:{emp}"): 'q0',
    ("palat", f"$:{emp}"): 'q0',

    ("length", "̄:̄"): "q0",  #checking for length
    ("length", "̆:̆"): "q0",
    ("length", "п:p"): 'palat',
    ("length", "т:t"): 'palat',
    ("length", "ч:č"): 'q0',
    ("length", "к:k"): 'palat',
    ("length", "м:m"): 'palat',
    ("length", "н:n"): 'palat',
    ("length", "ӈ:ŋ"): 'palat',
    ("length", "с:s"): 'palat',
    ("length", "ш:sʹ"): 'q0',
    ("length", "х:x"): 'palat',
    ("length", "ԯ:λ"): 'palat',
    ("length", "в:w"): 'palat',
    ("length", "л:l"): 'palat',
    ("length", "д:dʹ"): 'q0',
    ("length", f"':{emp}"): 'checkh',
    ("length", f'":{emp}'): 'checkh',
    
    ("length", "а:a"): 'length',
    ("length", "о:o"): 'length',
    ("length", "э:e"): 'length',
    ("length", "у:u"): 'length',
    ("length", "ы:i"): 'length',
    ("length", "ӹ:æ̆"): 'q0',
    ("length", "ӭ:æ"): 'q0',
    ("length", " : "): 'q0',

    ("checkh", "п:ʰp"): 'palat',  # checking ʔ/ʰ allomorphy
    ("checkh", "т:ʰt"): 'palat',
    ("checkh", "ч:ʰč"): 'q0',
    ("checkh", "к:ʰk"): 'palat',
    ("checkh", "м:ʔm"): 'palat',
    ("checkh", "н:ʔn"): 'palat',
    ("checkh", "ӈ:ʔŋ"): 'palat',
    ("checkh", "с:ʰs"): 'palat',
    ("checkh", "ш:ʰsʹ"): 'q0',
    ("checkh", "х:ʰx"): 'palat',
    ("checkh", "ԯ:ʰλ"): 'palat',
    ("checkh", "в:ʰw"): 'palat',
    ("checkh", "л:ʰl"): 'palat',
    ("checkh", "д:ʰdʹ"): 'q0',   
    
    ("checkh", "а:ʔa"): 'length',
    ("checkh", "о:ʔo"): 'length',
    ("checkh", "э:ʔe"): 'length',
    ("checkh", "у:ʔu"): 'length',
    ("checkh", "ы:ʔi"): 'length',
    ("checkh", "ӹ:ʔæ̆"): 'q0',
    ("checkh", "ӭ:ʔæ"): 'q0',
    ("checkh", " : "): 'q0',

    # state for putting in '#' in case there is an unknown symbol
    ("Error", f"%:{emp}"): 'q0',  # consonants return their corresponding Latin characters, checking most of them as palatilized
    ("Error", f"$:{emp}"): 'q0',
    ("Error", "п:p"): 'palat',
    ("Error", "т:t"): 'palat',
    ("Error", "ч:č"): 'q0',  # doesn't have a palatalized corresponding character
    ("Error", "к:k"): 'palat',
    ("Error", "м:m"): 'palat',
    ("Error", "н:n"): 'palat',
    ("Error", "ӈ:ŋ"): 'palat',
    ("Error", "с:s"): 'palat',
    ("Error", "ш:sʹ"): 'q0',  # doesn't have a palatalized corresponding character
    ("Error", "х:x"): 'palat',
    ("Error", "ԯ:λ"): 'palat',
    ("q0", "в:w"): 'palat',
    ("Error", "л:l"): 'palat',
    ("Error", "д:dʹ"): 'q0',  # doesn't have a velarized corresponding character
    ("Error", f"':{emp}"): 'checkh',  # cheking if there's a ʰ
    ("Error", f'":{emp}'): 'checkh',
    ("Error", "а:a"): 'length',  # checking most of the vowels for length; æ can't be long, it is mostly unmarked
    ("Error", "о:o"): 'length',
    ("Error", "э:e"): 'length',
    ("Error", "у:u"): 'length',
    ("Error", "ы:i"): 'length',
    ("Error", "ӹ:æ̆"): 'q0',
    ("Error", "ӭ:æ"): 'q0',
    ("Error", " : "): 'q0',  # making whitespaces
}

In [90]:
""" Функция переходов: берёт входное состояние и переходит в другое, выполняя операцию на ребре."""

def get_state(current_state, symbol):  
    for (src_state, program), tgt_state in transition.items():
        if src_state == current_state and program[0] == symbol:
            return tgt_state, program[2:]
    return "Error", "#"

In [91]:
"""Функция транслитерации: берёт строку, обрабатывает её и применяет к ней функцию get_state"""
def translit(string: str):
    start_state = "q0"
    newstring = re.sub(r"[^\w\s\'\"\̄ \̆ \ ]", '', string)
    inpstring = "%" + newstring.lower()
    outstring = ""
    current_state = start_state
    for x in inpstring:  
        current_state, to_print = get_state(current_state, x)
        outstring += to_print
    return outstring

#### Пример работы:

In [92]:
translit("Ма̄нӭкко")  # мешок

'mānækko'

In [93]:
translit('кали"та')  # он сам

'kalʹiʰta'

In [94]:
translit("кампуло'ш")  # намусорить, насорить

'kampuloʰsʹ'

In [95]:
translit('Мань маны"ӈам Петям, Васям')  # Я вижу Петю, Васю

'manʹ maniʔŋam pʹetʹam wasʹam'

In [96]:
translit('Ма!ь маqы"ӈам Пеzям, Ваiям')  # Я вижу Петю, Васю, с ошибками

'ma# ma#iʔŋam pʹe##m wa##m'